In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
import matplotlib.pyplot as plt

In [4]:
x = np.random.rand(4, 10, 8, 128)
y = keras.layers.Conv2DTranspose(32, 2, 2, activation='relu',)(x)
print(y.shape)

(4, 20, 16, 32)


In [5]:
def discriminator(in_shape = (28,28,1)):
    model  = Sequential()
    model.add(keras.Input(shape=in_shape))
    
    model.add(keras.layers.Conv2D(64,3,2,padding='same',activation='leaky_relu'))
    model.add(keras.layers.Dropout(0.3))
    
    model.add(keras.layers.Conv2D(128,3,2,padding='same',activation='leaky_relu'))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(256,activation='leaky_relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(),
                  metrics = ['accuracy'])

    return model

In [6]:
print(discriminator().summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 14, 14, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       1,605,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,680,641 (6.41 MB)

 Trainable params: 1,680,641 (6.41 MB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:
def generator(rv_dim = 100):
    model = Sequential()
    model.add(keras.Input(shape=(rv_dim,)))
    model.add(keras.layers.Dense(7*7*256,activation='leaky_relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Reshape((7,7,256)))

    model.add(keras.layers.Conv2DTranspose(128,3,2,padding='same',use_bias=False,activation = 'leaky_relu'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv2DTranspose(64,3,2,padding='same',use_bias=False,activation = 'leaky_relu'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv2DTranspose(1,3,1,padding='same',use_bias=False,activation = 'tanh'))

    model.compile(loss = 'mse',
                  optimizer=Adam(),
                  metrics=['accuracy'])

    return model
    

In [14]:
print(generator().summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                      │ (None, 12544)               │       1,266,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 12544)               │          50,176 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_3 (Reshape)                  │ (None, 7, 7, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_10                  │ (None, 14, 14, 128)         │         294,912 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 14, 14, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_11                  │ (None, 28, 28, 64)          │          73,728 │
│ (Conv2DTranspose)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_12                  │ (None, 28, 28, 1)           │             576 │
│ (Conv2DTranspose)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,687,104 (6.44 MB)

 Trainable params: 1,661,632 (6.34 MB)

 Non-trainable params: 25,472 (99.50 KB)

None


In [15]:
def gan_model(g_model,d_model):
    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(),
                  metrics = ['accuracy'])
    return model

In [16]:
print(gan_model(generator(),discriminator()).summary())

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)            │ (None, 28, 28, 1)           │       1,687,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_6 (Sequential)            │ (None, 1)                   │       1,680,641 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,367,745 (12.85 MB)

 Trainable params: 3,342,273 (12.75 MB)

 Non-trainable params: 25,472 (99.50 KB)

None


In [25]:
def load_real_data():
    (X_train,_),(_,_) = load_data()
    X_train = X_train.reshape((-1,28,28,1)).astype('f8')-127.5
    return X_train/127.5

In [18]:
def generate_real_samples(data,n_samples = 100):
    ix = np.random.randint(0,data.shape[0],n_samples)
    X_train = data[ix]
    y = np.ones(shape=(X_train.shape[0],1))
    return X_train,y

In [19]:
# data = load_real_data()
# generate_real_samples(data,1)

In [20]:
def generate_rv(rv_dim,n_sample=100):
    return np.random.randn(n_sample,rv_dim)

In [21]:
def generate_fake_images(g_model,rv_dim,n_samples = 100):
    rv = generate_rv(rv_dim,n_samples)
    fimg = g_model.predict(rv)
    y = np.zeros(shape = (n_samples,1))
    return fimg,y

In [27]:
def save_fig(g_model,rv_dim,epoch):
    n = 10
    rv = generate_rv(rv_dim,n*n)
    f_imgs = g_model.predict(rv)
    for i in range(n * n):
        plt.subplot(n, n, 1+i)
        plt.axis('off')
        plt.imshow(f_imgs[i].reshape((28,28)), interpolation='nearest',cmap = 'gray')
    filename = f'./DCGAN_output/generated_plot_e{epoch}.png'
    plt.savefig(filename)
    plt.close()

In [28]:
def train(data,g_model,d_model,gan_model,rv_dim,epochs = 51,batch_size = 256):
    nbatchs = data.shape[0]//batch_size
    half_batch = batch_size//2

    for e in range(epochs):
        for bn in range(nbatchs):
            x_real,y_real = generate_real_samples(data,half_batch)
            x_fake,y_fake = generate_fake_images(g_model,rv_dim,half_batch)

            d_model.trainable = True

            r_loss,_ = d_model.train_on_batch(x_real,y_real)
            f_loss,_ = d_model.train_on_batch(x_fake,y_fake)

            d_loss = 0.5*(r_loss+f_loss)

            d_model.trainable = False

            x_rv = generate_rv(rv_dim,batch_size)
            y = np.ones(shape=(x_rv.shape[0],1))

            g_loss,_ = gan_model.train_on_batch(x_rv,y)

        print(f'Epoch: {e+1}, d_loss: {d_loss}, g_loss: {g_loss}')
        if e%10 == 0:
            save_fig(g_model,rv_dim,e)

In [29]:
keras.utils.disable_interactive_logging()
rv_dim = 50
g_model = generator(rv_dim)
d_model = discriminator()
gan = gan_model(g_model,d_model)
data = load_real_data()
train(data,g_model,d_model,gan,rv_dim)
keras.utils.enable_interactive_logging()

Epoch: 1, d_loss: 0.00863610953092575, g_loss: 0.0012398454127833247
Epoch: 2, d_loss: 0.02248934656381607, g_loss: 0.7521854639053345
Epoch: 3, d_loss: 0.06369946151971817, g_loss: 1.6817976236343384
Epoch: 4, d_loss: 0.09210868924856186, g_loss: 2.2452476024627686
Epoch: 5, d_loss: 0.13529643416404724, g_loss: 2.4064242839813232
Epoch: 6, d_loss: 0.1658152937889099, g_loss: 2.49241304397583
Epoch: 7, d_loss: 0.1862463355064392, g_loss: 2.5405354499816895
Epoch: 8, d_loss: 0.20561394095420837, g_loss: 2.533799886703491
Epoch: 9, d_loss: 0.22295427322387695, g_loss: 2.511138916015625
Epoch: 10, d_loss: 0.2371928095817566, g_loss: 2.490116834640503
Epoch: 11, d_loss: 0.25031614303588867, g_loss: 2.465204954147339
Epoch: 12, d_loss: 0.2611803412437439, g_loss: 2.4499120712280273
Epoch: 13, d_loss: 0.2698724567890167, g_loss: 2.437155246734619
Epoch: 14, d_loss: 0.27877652645111084, g_loss: 2.411497116088867
Epoch: 15, d_loss: 0.28707772493362427, g_loss: 2.387601375579834
Epoch: 16, d_lo